# Trabalho 7 - Parte II

### Aluno: Gabriel Alessi Posonski

### RA: 2259583

### Instalação e importação de bibliotecas

In [1]:
#Descomentar caso precise instalar as bibliotecas
#%pip install pandas
#%pip install scikit-learn
#%pip install imbalanced-learn



In [2]:

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler

#### Carregar base e remover coluna ID

In [3]:
base = pd.read_excel('BaseDadosNotas.xlsx', sheet_name='Base Completa')
base = base.drop(columns=['ID'])
base.head()


,PROVA1,PROVA2,TRABALHO,SITUAÇÃO
0,2.0,1.5,0.5,Reprovado
1,8.0,4.8,8.8,Aprovado
2,3.0,0.0,1.8,Reprovado
3,8.0,7.5,2.3,Aprovado
4,2.5,0.0,1.2,Reprovado


#### Codificação da Coluna SITUAÇÃO

In [4]:
base['SITUAÇÃO'] = base['SITUAÇÃO'].map({'Reprovado': 0, 'Aprovado': 1})
base.head()

,PROVA1,PROVA2,TRABALHO,SITUAÇÃO
0,2.0,1.5,0.5,0
1,8.0,4.8,8.8,1
2,3.0,0.0,1.8,0
3,8.0,7.5,2.3,1
4,2.5,0.0,1.2,0


#### Normalização

In [5]:
# Normalização por MinMax
scaler = MinMaxScaler()
base[['PROVA1', 'PROVA2','TRABALHO']] = scaler.fit_transform(base[['PROVA1', 'PROVA2','TRABALHO']])

base.head()

,PROVA1,PROVA2,TRABALHO,SITUAÇÃO
0,0.20,0.15,0.05,0
1,0.80,0.48,0.88,1
2,0.30,0.00,0.18,0
3,0.80,0.75,0.23,1
4,0.25,0.00,0.12,0


#### Balanceamento de classes

In [6]:
# Verificar a distribuição das classes
base['SITUAÇÃO'].value_counts(normalize=True)

SITUAÇÃO
1    0.696833
0    0.303167
Name: proportion, dtype: float64

A proporção de aprovados é de quase 70%, aplicaremos um balanceamento

In [7]:
# balanceamento da base de dados
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X = base.drop('SITUAÇÃO', axis=1)
y = base['SITUAÇÃO']
X_res, y_res = rus.fit_resample(X, y)
base_balanceada = pd.concat([pd.DataFrame(X_res, columns=X.columns), pd.DataFrame(y_res, columns=['SITUAÇÃO'])], axis=1)
base_balanceada['SITUAÇÃO'].value_counts(normalize=True)
base_balanceada.head()
base_balanceada.info()


<class 'pandas.core.frame.DataFrame'>
Index: 134 entries, 0 to 113
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   PROVA1    134 non-null    float64
 1   PROVA2    134 non-null    float64
 2   TRABALHO  134 non-null    float64
 3   SITUAÇÃO  134 non-null    int64  
dtypes: float64(3), int64(1)
memory usage: 5.2 KB


Com o balanceamento feito, podemos prosseguir para o treinamento

#### Divisão estratificada e salvamento dos dados

In [8]:
from sklearn.model_selection import train_test_split

# Divisão estratificada 2/3 treinamento e 1/3 teste
X = base_balanceada.drop('SITUAÇÃO', axis=1)
y = base_balanceada['SITUAÇÃO']

# Divisão estratificada mantendo o balanceamento das classes
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=1/3,  # 1/3 para teste
    random_state=42, 
    stratify=y  # Mantém o balanceamento das classes
)

print("Distribuição das classes no conjunto de treinamento:")
print(y_train.value_counts(normalize=True))
print("\nDistribuição das classes no conjunto de teste:")
print(y_test.value_counts(normalize=True))

# Criar DataFrames completos para salvar
dados_treinamento = pd.concat([X_train, y_train], axis=1)
dados_teste = pd.concat([X_test, y_test], axis=1)

# Salvar os arquivos com header correto para txt
dados_treinamento.to_csv('treinamento.txt', index=False, sep='\t', header=True)
dados_teste.to_csv('teste.txt', index=False, sep='\t', header=True)

print(f"\nTamanho do conjunto de treinamento: {len(dados_treinamento)} amostras")
print(f"Tamanho do conjunto de teste: {len(dados_teste)} amostras")
print("\nArquivos salvos:")
print("- treinamento.txt")
print("- teste.txt")

# Verificar se os arquivos foram salvos corretamente
print("\nVerificação dos arquivos salvos:")
test_load = pd.read_csv('treinamento.txt', sep='\t')
print("Colunas no arquivo:", test_load.columns.tolist())
print("Primeiras linhas:")
print(test_load.head())

Distribuição das classes no conjunto de treinamento:
SITUAÇÃO
1    0.505618
0    0.494382
Name: proportion, dtype: float64

Distribuição das classes no conjunto de teste:
SITUAÇÃO
0    0.511111
1    0.488889
Name: proportion, dtype: float64

Tamanho do conjunto de treinamento: 89 amostras
Tamanho do conjunto de teste: 45 amostras

Arquivos salvos:
- treinamento.txt
- teste.txt

Verificação dos arquivos salvos:
Colunas no arquivo: ['PROVA1', 'PROVA2', 'TRABALHO', 'SITUAÇÃO']
Primeiras linhas:
   PROVA1  PROVA2  TRABALHO  SITUAÇÃO
0   0.750    0.50      0.33         0
1   0.938    1.00      0.95         1
2   0.938    0.90      1.00         1
3   0.750    0.95      0.21         1
4   0.800    1.00      0.33         1


### Treinamento com Múltiplas Configurações de Rede

In [9]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import pandas as pd

# Recarregar os dados divididos
dados_treinamento = pd.read_csv('treinamento.txt', sep='\t')
dados_teste = pd.read_csv('teste.txt', sep='\t')

X_train = dados_treinamento.drop('SITUAÇÃO', axis=1)
y_train = dados_treinamento['SITUAÇÃO']
X_test = dados_teste.drop('SITUAÇÃO', axis=1)
y_test = dados_teste['SITUAÇÃO']

# Configurações de rede a serem testadas
configuracoes = {
    'Configuração 1': (5,),      # 1 camada oculta com 5 neurônios
    'Configuração 2': (10,),     # 1 camada oculta com 10 neurônios
    'Configuração 3': (5, 3),    # 2 camadas ocultas (5 e 3 neurônios)
    'Configuração 4': (10, 5),   # 2 camadas ocultas (10 e 5 neurônios)
}

# Épocas a serem testadas
epocas = [30, 50, 100]

# Armazenar resultados
resultados = []

print("=== TREINAMENTO COM MÚLTIPLAS CONFIGURAÇÕES ===\n")

for nome_config, camadas in configuracoes.items():
    print(f"--- {nome_config}: {camadas} ---")
    
    for epoca in epocas:
        print(f"  Épocas: {epoca}")
        
        # Criar e treinar o modelo
        mlp = MLPClassifier(
            hidden_layer_sizes=camadas,
            max_iter=epoca,
            random_state=42,
            solver='adam',
            learning_rate_init=0.01
        )
        
        # Treinar o modelo
        mlp.fit(X_train, y_train)
        
        # Fazer predições
        y_pred_train = mlp.predict(X_train)
        y_pred_test = mlp.predict(X_test)
        
        # Calcular acurácias
        acc_train = accuracy_score(y_train, y_pred_train)
        acc_test = accuracy_score(y_test, y_pred_test)
        
        # Armazenar resultados
        resultado = {
            'Configuração': nome_config,
            'Camadas': str(camadas),
            'Épocas': epoca,
            'Acurácia Treino': acc_train,
            'Acurácia Teste': acc_test,
            'Convergiu': mlp.n_iter_ < epoca
        }
        resultados.append(resultado)
        
        print(f"    Treino: {acc_train:.4f} | Teste: {acc_test:.4f} | Convergiu: {mlp.n_iter_ < epoca}")
    
    print()

# Converter resultados para DataFrame para melhor visualização
df_resultados = pd.DataFrame(resultados)
print("=== RESUMO DOS RESULTADOS ===")
print(df_resultados.to_string(index=False))

=== TREINAMENTO COM MÚLTIPLAS CONFIGURAÇÕES ===

--- Configuração 1: (5,) ---
  Épocas: 30
    Treino: 0.5955 | Teste: 0.5111 | Convergiu: False
  Épocas: 50
    Treino: 0.7640 | Teste: 0.7333 | Convergiu: False
  Épocas: 100
    Treino: 0.8539 | Teste: 0.8000 | Convergiu: False

--- Configuração 2: (10,) ---
  Épocas: 30
    Treino: 0.7865 | Teste: 0.7333 | Convergiu: False
  Épocas: 50
    Treino: 0.9101 | Teste: 0.9111 | Convergiu: False
  Épocas: 100
    Treino: 0.9775 | Teste: 0.9556 | Convergiu: False

--- Configuração 3: (5, 3) ---
  Épocas: 30
    Treino: 0.5056 | Teste: 0.4889 | Convergiu: False
  Épocas: 50
    Treino: 0.5056 | Teste: 0.4889 | Convergiu: True
  Épocas: 100
    Treino: 0.5056 | Teste: 0.4889 | Convergiu: True

--- Configuração 4: (10, 5) ---
  Épocas: 30
    Treino: 0.5056 | Teste: 0.4889 | Convergiu: True

--- Configuração 4: (10, 5) ---
  Épocas: 30
    Treino: 0.8652 | Teste: 0.8667 | Convergiu: False
  Épocas: 50
    Treino: 0.9101 | Teste: 0.8889 | Conver

c:\Users\Gabriel\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Gabriel\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Gabriel\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Gabriel\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the

    Treino: 0.9888 | Teste: 0.9556 | Convergiu: False

=== RESUMO DOS RESULTADOS ===
  Configuração Camadas  Épocas  Acurácia Treino  Acurácia Teste  Convergiu
Configuração 1    (5,)      30         0.595506        0.511111      False
Configuração 1    (5,)      50         0.764045        0.733333      False
Configuração 1    (5,)     100         0.853933        0.800000      False
Configuração 2   (10,)      30         0.786517        0.733333      False
Configuração 2   (10,)      50         0.910112        0.911111      False
Configuração 2   (10,)     100         0.977528        0.955556      False
Configuração 3  (5, 3)      30         0.505618        0.488889      False
Configuração 3  (5, 3)      50         0.505618        0.488889       True
Configuração 3  (5, 3)     100         0.505618        0.488889       True
Configuração 4 (10, 5)      30         0.865169        0.866667      False
Configuração 4 (10, 5)      50         0.910112        0.888889      False
Configuração 4 

c:\Users\Gabriel\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


#### Análise da Melhor Configuração

In [10]:
# Encontrar a melhor configuração baseada na acurácia de teste
melhor_resultado = df_resultados.loc[df_resultados['Acurácia Teste'].idxmax()]

print("=== MELHOR CONFIGURAÇÃO ===")
print(f"Configuração: {melhor_resultado['Configuração']}")
print(f"Camadas: {melhor_resultado['Camadas']}")
print(f"Épocas: {melhor_resultado['Épocas']}")
print(f"Acurácia de Teste: {melhor_resultado['Acurácia Teste']:.4f}")
print(f"Acurácia de Treino: {melhor_resultado['Acurácia Treino']:.4f}")
print()

# Treinar novamente a melhor configuração para análise detalhada
melhor_config = eval(melhor_resultado['Camadas'])
melhor_epocas = melhor_resultado['Épocas']

mlp_melhor = MLPClassifier(
    hidden_layer_sizes=melhor_config,
    max_iter=melhor_epocas,
    random_state=42,
    solver='adam',
    learning_rate_init=0.01
)

mlp_melhor.fit(X_train, y_train)
y_pred_melhor = mlp_melhor.predict(X_test)

# Matriz de confusão
print("=== MATRIZ DE CONFUSÃO (MELHOR MODELO) ===")
conf_matrix = confusion_matrix(y_test, y_pred_melhor)
print(conf_matrix)
print()

# Relatório de classificação
print("=== RELATÓRIO DE CLASSIFICAÇÃO ===")
print(classification_report(y_test, y_pred_melhor, target_names=['Reprovado', 'Aprovado']))

# Calcular importância das variáveis para o melhor modelo
weights_input_hidden = mlp_melhor.coefs_[0]

# Calcular a importância absoluta média de cada variável
importancia_prova1 = np.mean(np.abs(weights_input_hidden[0, :]))
importancia_prova2 = np.mean(np.abs(weights_input_hidden[1, :]))
importancia_trabalho = np.mean(np.abs(weights_input_hidden[2, :]))

# Calcular pesos proporcionais que somem 10 (permitindo decimais)
soma_importancias = importancia_prova1 + importancia_prova2 + importancia_trabalho

# Distribuir os 10 pontos proporcionalmente com precisão decimal
peso_prova1 = round((importancia_prova1 / soma_importancias) * 10, 2)
peso_prova2 = round((importancia_prova2 / soma_importancias) * 10, 2)
peso_trabalho = round((importancia_trabalho / soma_importancias) * 10, 2)

# Verificar e ajustar para garantir que a soma seja exatamente 10.00
soma_atual = peso_prova1 + peso_prova2 + peso_trabalho
diferenca = round(10.0 - soma_atual, 2)

# Se houver diferença de arredondamento, ajustar a variável com maior importância
if diferenca != 0:
    importancias_list = [importancia_prova1, importancia_prova2, importancia_trabalho]
    indice_maior = importancias_list.index(max(importancias_list))
    
    if indice_maior == 0:
        peso_prova1 = round(peso_prova1 + diferenca, 2)
    elif indice_maior == 1:
        peso_prova2 = round(peso_prova2 + diferenca, 2)
    else:
        peso_trabalho = round(peso_trabalho + diferenca, 2)

print("\n=== IMPORTÂNCIA DAS VARIÁVEIS (Total = 10.0) ===")
print(f"prova 1: peso {peso_prova1}")
print(f"prova 2: peso {peso_prova2}")
print(f"trabalho: peso {peso_trabalho}")
print(f"Total: {peso_prova1 + peso_prova2 + peso_trabalho}")

# Mostrar também os valores brutos para referência
print(f"\n=== VALORES BRUTOS DE IMPORTÂNCIA ===")
print(f"PROVA 1: {importancia_prova1:.6f}")
print(f"PROVA 2: {importancia_prova2:.6f}")
print(f"TRABALHO: {importancia_trabalho:.6f}")
print(f"Soma: {soma_importancias:.6f}")

# Análise adicional
pesos = [peso_prova1, peso_prova2, peso_trabalho]
variaveis = ['prova 1', 'prova 2', 'trabalho']
mais_importante = variaveis[pesos.index(max(pesos))]

print(f"\nVariável mais importante: {mais_importante}")

=== MELHOR CONFIGURAÇÃO ===
Configuração: Configuração 2
Camadas: (10,)
Épocas: 100
Acurácia de Teste: 0.9556
Acurácia de Treino: 0.9775

=== MATRIZ DE CONFUSÃO (MELHOR MODELO) ===
[[21  2]
 [ 0 22]]

=== RELATÓRIO DE CLASSIFICAÇÃO ===
              precision    recall  f1-score   support

   Reprovado       1.00      0.91      0.95        23
    Aprovado       0.92      1.00      0.96        22

    accuracy                           0.96        45
   macro avg       0.96      0.96      0.96        45
weighted avg       0.96      0.96      0.96        45


=== IMPORTÂNCIA DAS VARIÁVEIS (Total = 10.0) ===
prova 1: peso 3.12
prova 2: peso 3.98
trabalho: peso 2.9
Total: 10.0

=== VALORES BRUTOS DE IMPORTÂNCIA ===
PROVA 1: 0.527825
PROVA 2: 0.672143
TRABALHO: 0.489190
Soma: 1.689158

Variável mais importante: prova 2


c:\Users\Gabriel\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
